This notebook containes code that is used to add doi and paragraph of software mentions from CZI linked data set. 

In [4]:
import pandas as pd

In [5]:
 
#Read files
labels = pd.read_csv("../CZI/comm_IDs_with_labels.csv",delimiter="\t")


In [6]:
print(labels.head())

  license                      location    pmcid  pmid  \
0    comm  comm/Micropl/PMC8475362.nxml  8475362   NaN   
1    comm  comm/Micropl/PMC8475362.nxml  8475362   NaN   
2    comm  comm/Micropl/PMC8475362.nxml  8475362   NaN   
3    comm  comm/Micropl/PMC8475362.nxml  8475362   NaN   
4    comm  comm/Micropl/PMC8475362.nxml  8475362   NaN   

                          doi  pubdate  \
0  10.1186/s43591-021-00017-9     2021   
1  10.1186/s43591-021-00017-9     2021   
2  10.1186/s43591-021-00017-9     2021   
3  10.1186/s43591-021-00017-9     2021   
4  10.1186/s43591-021-00017-9     2021   

                                              source  number  \
0  Particle selection and identification of polym...       7   
1  Particle selection and identification of polym...       8   
2                               Statistical analysis      12   
3                               Statistical analysis      12   
4                               Statistical analysis      12   

             

In [8]:
pypi = pd.read_csv("../CZI/pypi_df.csv")

In [4]:
github = pd.read_csv("../CZI/github.csv",encoding="utf-16")

In [7]:
cran = pd.read_csv("../CZI/cran_df.csv")
print(cran.columns)
print(cran.shape)
print(cran.head())


Index(['ID', 'software_mention', 'mapped_to', 'source', 'package_url',
       'homepage_url', 'other_urls', 'license', 'github_repo',
       'github_repo_license', 'exact_match'],
      dtype='object')
(6234, 11)
         ID software_mention    mapped_to      source  \
0   SM26805               A3           A3  CRAN Index   
1   SM40389           ABACUS       ABACUS  CRAN Index   
2  SM291013              abc          abc  CRAN Index   
3  SM538227      ABCanalysis  ABCanalysis  CRAN Index   
4  SM253680         abcdeFBA     abcdeFBA  CRAN Index   

                                         package_url  \
0  https://cran.r-project.org/web/packages/A3/ind...   
1  https://cran.r-project.org/web/packages/ABACUS...   
2  https://cran.r-project.org/web/packages/abc/in...   
3  https://cran.r-project.org/web/packages/ABCana...   
4  https://cran.r-project.org/web/packages/abcdeF...   

                                        homepage_url  other_urls    license  \
0                           

In [10]:
def add_doi_and_paragraph(df:pd.DataFrame)->pd.DataFrame:
    """
    Given:
      - df: your original metadata DataFrame with columns 
        ['ID','software_mention','mapped_to','source','platform',
         'package_url','homepage_url','other_urls','github_repo','exact_match']
      - labels: the “raw” mentions DataFrame (e.g. from comm_raw.tsv.gz)
        which must contain at least ['ID','doi','text'] columns

    Returns a new DataFrame where for each row in df you look up
    all matching (doi, text) pairs in labels.  If an ID has multiple
    matches in labels, you get additional rows (one per pair).  
    The returned DataFrame has all original df columns plus:
      - doi
      - paragraph  (renamed from labels.text)
    """
    # Extract only the needed columns from labels and rename text→paragraph
    label_subset = labels[['ID', 'doi', 'text']].rename(columns={'text': 'paragraph'})
    
    # Perform a left-join so every df row is preserved; rows with no match get NaN
    enriched = df.merge(label_subset, on='ID', how='left')
    enriched = enriched.dropna(subset=['doi', 'paragraph'])
    enriched = enriched.drop_duplicates(subset=['ID', 'doi', 'paragraph'], keep='first')

    # Reorder columns: original df cols + doi + paragraph
    original_cols = ['ID','software_mention','mapped_to',
                     'package_url','homepage_url','other_urls','github_repo','exact_match']
    return enriched[ original_cols + ['doi','paragraph'] ]

In [6]:
github=add_doi_and_paragraph(github)
github.to_csv("../github_from_CZI.csv")

In [9]:
pypi=add_doi_and_paragraph(pypi)
pypi.to_csv("../pypi_from_CZI.csv")

In [11]:
cran = add_doi_and_paragraph(cran)
cran.to_csv("../cran_from_CZI.csv")